# Introduction to Machine Learning

This notebook is an example of a CNN for recognizing handwritten characters.

Most of this code is from https://keras.io/examples/vision/mnist_convnet/

## Setup

In [1]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

## Prepare the data

In [2]:
# Model / data parameters
num_classes = 10
input_shape = (28, 28, 1)

# Load the data and split it between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")


# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


## Build the Model

In [3]:
model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 5, 5, 64)         0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 1600)              0         
                                                                 
 dropout (Dropout)           (None, 1600)              0

## Train the Model

In [4]:
batch_size = 128
epochs = 15

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

Epoch 1/15
422/422 [==============================] - 4s 3ms/step - loss: 0.3568 - accuracy: 0.8931 - val_loss: 0.0871 - val_accuracy: 0.9762
Epoch 2/15
422/422 [==============================] - 1s 4ms/step - loss: 0.1173 - accuracy: 0.9641 - val_loss: 0.0613 - val_accuracy: 0.9830
Epoch 3/15
422/422 [==============================] - 1s 4ms/step - loss: 0.0867 - accuracy: 0.9727 - val_loss: 0.0484 - val_accuracy: 0.9878
Epoch 4/15
422/422 [==============================] - 2s 4ms/step - loss: 0.0734 - accuracy: 0.9776 - val_loss: 0.0438 - val_accuracy: 0.9885
Epoch 5/15
422/422 [==============================] - 2s 4ms/step - loss: 0.0643 - accuracy: 0.9800 - val_loss: 0.0399 - val_accuracy: 0.9883
Epoch 6/15
422/422 [==============================] - 2s 5ms/step - loss: 0.0585 - accuracy: 0.9812 - val_loss: 0.0356 - val_accuracy: 0.9898
Epoch 7/15
422/422 [==============================] - 1s 3ms/step - loss: 0.0540 - accuracy: 0.9828 - val_loss: 0.0375 - val_accuracy: 0.9895
Epoch 

## Evaluate the Trained Model

In [5]:
score = model.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

Test loss: 0.024188049137592316
Test accuracy: 0.9922000169754028


## Example GUI

In [11]:
from tkinter import *
from PIL import ImageGrab
import imageio
import tkinter.font as font

class Paint(object):
    def __init__(self):
        self.root=Tk()
        self.root.title('Playing with numbers')
        # self.root.wm_iconbitmap('44143.ico')
        self.root.configure(background='light salmon')
        self.c = Canvas(self.root,bg='light cyan', height=330, width=400)
        self.label = Label(self.root, text='Draw any numer', font=20, bg='light salmon')
        self.label.grid(row=0, column=3)
        self.c.grid(row=1, columnspan=9)
        self.c.create_line(0,0,400,0,width=20,fill='midnight blue')
        self.c.create_line(0,0,0,330,width=20,fill='midnight blue')
        self.c.create_line(400,0,400,330,width=20,fill='midnight blue')
        self.c.create_line(0,330,400,330,width=20,fill='midnight blue')
        self.myfont = font.Font(size=20,weight='bold')
        self.predicting_button=Button(self.root,text='Predict', fg='white', bg='blue', height=2, width=6, font=self.myfont, command=lambda:self.classify(self.c))
        self.predicting_button.grid(row=2,column=1)
        self.clear=Button(self.root,text='Clear', fg='white', bg='orange', height=2, width=6, font=self.myfont, command=self.clear)
        self.clear.grid(row=2,column=5)
        self.prediction_text = Text(self.root, height=5, width=5)
        self.prediction_text.grid(row=4, column=3)
        self.label=Label(self.root, text="Predicted Number is", fg="black", font=30, bg='light salmon')

        self.label.grid(row=3,column=3)
        self.model=model
        self.setup()
        self.root.mainloop()


    def setup(self):
        self.old_x=None
        self.old_y=None
        self.color='black'
        self.linewidth=15
        self.c.bind('<B1-Motion>', self.paint)
        self.c.bind('<ButtonRelease-1>', self.reset)


    def paint(self,event):
        paint_color=self.color
        if self.old_x and self.old_y:
            self.c.create_line(self.old_x,self.old_y,event.x,event.y,fill=paint_color,width=self.linewidth,capstyle=ROUND,
                              smooth=TRUE,splinesteps=48)
        self.old_x=event.x
        self.old_y=event.y


    def clear(self):
        """Clear drawing area"""
        self.c.delete("all")

    def reset(self, event):
        """reset old_x and old_y if the left mouse button is released"""
        self.old_x, self.old_y = None, None


    def classify(self,widget):
        x=self.root.winfo_rootx()+widget.winfo_x()
        y=self.root.winfo_rooty()+widget.winfo_y()
        x1=widget.winfo_width()
        y1=widget.winfo_height()
        ImageGrab.grab().crop((x,y,x1,y1)).resize((28,28)).save('classify.png')
        img=imageio.imread('classify.png', as_gray=True, pilmode='P')
        img=np.array(img)
        img=np.reshape(img,(1,28,28,1))
        img[img==0] = 255
        img[img==225] = 0
        # Predict digit
        pred = self.model.predict([img])
        # Get index with highest probability
        pred = np.argmax(pred)
        print(pred)
        self.prediction_text.delete("1.0", END)
        self.prediction_text.insert(END, pred)
        labelfont = ('times', 30, 'bold')
        self.prediction_text.config(font=labelfont)

if __name__ == '__main__':
    Paint()

4
